# Regression of Used Car Prices First try

In [3]:
# Import the libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as pls
%matplotlib inline

In [59]:
pd.set_option('display.max_columns', None)

In [4]:
# Lets import the data which is available in the train data
data=pd.read_csv('train.csv')

In [5]:
data.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [6]:
data.shape

(188533, 13)

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            188533 non-null  int64 
 1   brand         188533 non-null  object
 2   model         188533 non-null  object
 3   model_year    188533 non-null  int64 
 4   milage        188533 non-null  int64 
 5   fuel_type     183450 non-null  object
 6   engine        188533 non-null  object
 7   transmission  188533 non-null  object
 8   ext_col       188533 non-null  object
 9   int_col       188533 non-null  object
 10  accident      186081 non-null  object
 11  clean_title   167114 non-null  object
 12  price         188533 non-null  int64 
dtypes: int64(4), object(9)
memory usage: 18.7+ MB


In [8]:
# Lets check null values
data.isnull().sum()

id                  0
brand               0
model               0
model_year          0
milage              0
fuel_type        5083
engine              0
transmission        0
ext_col             0
int_col             0
accident         2452
clean_title     21419
price               0
dtype: int64

In [15]:
# Lets check for the percentage of the missing values
features_with_nan=[features for features in data.columns if data[features].isnull().sum() > 1]
features_with_nan
for features in features_with_nan:
    print(features, np.round(data[features].isnull().mean(), 4), ' % missing values')

fuel_type 0.027  % missing values
accident 0.013  % missing values
clean_title 0.1136  % missing values


## Lets divide the data into features and label

In [20]:
data['fuel_type']=data['fuel_type'].fillna(data['fuel_type'].mode()[0])

In [21]:
data['accident']=data['accident'].fillna(data['accident'].mode()[0])

In [22]:
data['clean_title']=data['clean_title'].fillna(data['accident'].mode()[0])

In [23]:
data.isnull().sum()

id              0
brand           0
model           0
model_year      0
milage          0
fuel_type       0
engine          0
transmission    0
ext_col         0
int_col         0
accident        0
clean_title     0
price           0
dtype: int64

### Now the data is complete

In [24]:
#Lets drop the Id column
data.drop('id', axis=1, inplace=True)

In [25]:
data.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [28]:
# Lets check for the Cateogorical features in the dataset

cat_features=[features for features in data.columns if data[features].dtype == 'O']
cat_features

['brand',
 'model',
 'fuel_type',
 'engine',
 'transmission',
 'ext_col',
 'int_col',
 'accident',
 'clean_title']

In [32]:
data['brand'].value_counts()

brand
Ford             23088
Mercedes-Benz    19172
BMW              17028
Chevrolet        16335
Audi             10887
Porsche          10612
Land              9525
Toyota            8850
Lexus             8643
Jeep              6474
Cadillac          4674
RAM               4249
Nissan            3930
Tesla             3738
INFINITI          3276
GMC               3215
Dodge             3133
Mazda             2719
Kia               2497
Lincoln           2423
Subaru            2381
Acura             2282
Honda             2101
Hyundai           2045
Volkswagen        1765
Jaguar            1319
Bentley           1155
MINI              1064
Genesis            969
Buick              940
Maserati           939
Lamborghini        809
Chrysler           727
Volvo              723
Alfa               682
Rivian             590
Rolls-Royce        561
Mitsubishi         551
Pontiac            538
Hummer             520
Ferrari            359
McLaren            243
Aston              238
Satur

In [33]:
for features in cat_features:
    print(data[features].value_counts())

brand
Ford             23088
Mercedes-Benz    19172
BMW              17028
Chevrolet        16335
Audi             10887
Porsche          10612
Land              9525
Toyota            8850
Lexus             8643
Jeep              6474
Cadillac          4674
RAM               4249
Nissan            3930
Tesla             3738
INFINITI          3276
GMC               3215
Dodge             3133
Mazda             2719
Kia               2497
Lincoln           2423
Subaru            2381
Acura             2282
Honda             2101
Hyundai           2045
Volkswagen        1765
Jaguar            1319
Bentley           1155
MINI              1064
Genesis            969
Buick              940
Maserati           939
Lamborghini        809
Chrysler           727
Volvo              723
Alfa               682
Rivian             590
Rolls-Royce        561
Mitsubishi         551
Pontiac            538
Hummer             520
Ferrari            359
McLaren            243
Aston              238
Satur

In [35]:
data['engine'].value_counts()

engine
355.0HP 5.3L 8 Cylinder Engine Gasoline Fuel           3462
240.0HP 2.0L 4 Cylinder Engine Gasoline Fuel           2902
420.0HP 6.2L 8 Cylinder Engine Gasoline Fuel           2841
2.0L I4 16V GDI DOHC Turbo                             2680
375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel          2451
                                                       ... 
151.0HP 1.5L 4 Cylinder Engine Gas/Electric Hybrid        1
184.0HP 2.4L 4 Cylinder Engine Flex Fuel Capability       1
78.0HP 1.2L 3 Cylinder Engine Gasoline Fuel               1
139.0HP 1.6L 4 Cylinder Engine Plug-In Electric/Gas       1
313.0HP 2.0L 4 Cylinder Engine Plug-In Electric/Gas       1
Name: count, Length: 1117, dtype: int64

In [41]:
data_one_hot = pd.get_dummies(data, columns=cat_features, drop_first=True)

In [60]:
data_one_hot

model_year  milage  price  brand_Alfa  brand_Aston  brand_Audi  \
0             2007  213000   4200       False        False       False   
1             2002  143250   4999       False        False       False   
2             2002  136731  13900       False        False       False   
3             2017   19500  45000       False        False       False   
4             2021    7388  97500       False        False       False   
...            ...     ...    ...         ...          ...         ...   
188528        2017   49000  27500       False        False       False   
188529        2018   28600  30000       False        False       False   
188530        2021   13650  86900       False        False       False   
188531        2022   13895  84900       False        False        True   
188532        2016   59500  28995       False        False       False   

        brand_BMW  brand_Bentley  brand_Bugatti  brand_Buick  brand_Cadillac  \
0           False          False          False        False           False   
1           False          False          False        False           False   
2           False          False          False        False           False   
3           False          False          False        False           False   
4           False          False          False        False           False   
...           ...            ...            ...          ...             ...   
188528      False          False          False        False            True   
188529      False          False          False        False           False   
188530      False          False          False        False           False   
188531      False          False          False        False           False   
188532      False          False          False        False           False   

        brand_Chevrolet  brand_Chrysler  brand_Dodge  brand_FIAT  \
0                 False           False        False       False   
1                 False           False        False       False   
2                  True           False        False       False   
3                 False           False        False       False   
4                 False           False        False       False   
...                 ...             ...          ...         ...   
188528            False           False        False       False   
188529            False           False        False       False   
188530            False           False        False       False   
188531            False           False        False       False   
188532            False           False        False       False   

        brand_Ferrari  brand_Ford  brand_GMC  brand_Genesis  brand_Honda  \
0               False       False      False          False        False   
1               False       False      False          False        False   
2               False       False      False          False        False   
3               False       False      False           True        False   
4               False       False      False          False        False   
...               ...         ...        ...            ...          ...   
188528          False       False      False          False        False   
188529          False       False      False          False        False   
188530          False       False      False          False        False   
188531          False       False      False          False        False   
188532          False       False      False          False        False   

        brand_Hummer  brand_Hyundai  brand_INFINITI  brand_Jaguar  brand_Jeep  \
0              False          False           False         False       False   
1              False          False           False         False       False   
2              False          False           False         False       False   
3              False          False           False         False       False   
4              False   

In [63]:
features=data_one_hot.drop('price', axis=1).values
label=data_one_hot['price'].values

In [64]:
features

array([[2007, 213000, False, ..., False, True, True],
       [2002, 143250, False, ..., False, False, True],
       [2002, 136731, False, ..., False, True, True],
       ...,
       [2021, 13650, False, ..., False, True, True],
       [2022, 13895, False, ..., False, True, False],
       [2016, 59500, False, ..., False, True, True]], dtype=object)

In [65]:
label

array([ 4200,  4999, 13900, ..., 86900, 84900, 28995], dtype=int64)

# Lets build the model

### Linear Regression Model

In [69]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(features,
                                                label,
                                                test_size=0.3,
                                                random_state=42)

MemoryError: Unable to allocate 3.54 GiB for an array with shape (131973, 3602) and data type object

In [70]:
# Randomly sample 50% of the data (or choose a suitable percentage)
data_sampled = data_one_hot.sample(frac=0.5, random_state=42)

# Split the smaller data
X_sampled = data_sampled.drop('price', axis=1)
y_sampled = data_sampled['price']

X_train, X_test, y_train, y_test = train_test_split(X_sampled, y_sampled, test_size=0.2, random_state=42)


In [71]:
from sklearn.linear_model import LinearRegression
model=LinearRegression()
model.fit(X_train, y_train)

MemoryError: Unable to allocate 2.02 GiB for an array with shape (3602, 75412) and data type int64

In [72]:
sns.pairplot(data_one_hot)

Unexpected exception formatting exception. Falling back to standard exception
Error in callback <function _draw_all_if_interactive at 0x000002A94A870900> (for post_execute), with arguments args (),kwargs {}:


Traceback (most recent call last):
  File "C:\Users\cheta\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\cheta\AppData\Local\Temp\ipykernel_7652\2562677992.py", line 1, in <module>
    sns.pairplot(data_one_hot)
  File "C:\Users\cheta\anaconda3\Lib\site-packages\seaborn\axisgrid.py", line 2114, in pairplot
    grid = PairGrid(data, vars=vars, x_vars=x_vars, y_vars=y_vars, hue=hue,
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\cheta\anaconda3\Lib\site-packages\seaborn\axisgrid.py", line 1276, in __init__
    axes = fig.subplots(len(y_vars), len(x_vars),
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\cheta\anaconda3\Lib\site-packages\matplotlib\figure.py", line 931, in subplots
    axs = gs.subplots(sharex=sharex, sharey=sharey, squeeze=squeeze,
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


ValueError: Image size of 900750x900750 pixels is too large. It must be less than 2^16 in each direction.

ValueError: Image size of 900750x900750 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 900750x900750 with 2217 Axes>

As you can see here we have encountered with problem of memory, the reason being done here is to show as how creating getdummies using the pandas 
has created this issue, the reason here it has created lot of columns, which resulted into creating multiple columns, so when we are feeding
these many columns to the model it is creating the memory issue.

This issue we will try to resolve with the help of other encoding techniques for categorical data.